### define function extract_spaces_with_area
this function extracts important information about all the ifc spaces in an architectural model. 
for each space, it will retrieve its unique id, name (long name if available), and floor area. 
it also identifies the source of the area, whether it comes from the standard ifc quantity set 
(Qto_SpaceBaseQuantities) or a fallback dimensions property set. 
the function returns a list of dictionaries, one for each space.


In [ ]:
def extract_spaces_with_area(arch_model, *, verbose=True):
    """
    Extracts area, name, and IFC ID for each IfcSpace in an IFC model.

    Parameters
    ----------
    arch_model : ifcopenshell.file
        The ARCH model (or any IFC file containing IfcSpace).
    verbose : bool
        If True, prints each space as in your original snippet.

    Returns
    -------
    list of dict
        [
            {
                "id": int,
                "name": str,
                "area": float or None,
                "source": "Qto_SpaceBaseQuantities" | "Dimensions" | None
            },
            ...
        ]
    """
    results = []
    Spaces = arch_model.by_type("IfcSpace")


### iterate over spaces
loop through each IfcSpace object in the model. 
for every space, retrieve the property sets (psets) associated with it and initialize 
variables to store the area, source of area, and type of area measurement. 
these will be updated in the next steps depending on what data is available for the space.


In [ ]:
    for space in Spaces:
        psets = uel.get_psets(space)
        area = None
        source = None
        area_type = None


### check preferred quantity set
check if the standard IFC quantity set "Qto_SpaceBaseQuantities" is present. 
if available, try to get the net floor area first. if net floor area is not available, 
attempt to use gross floor area as a fallback. 
this ensures that the most reliable IFC-based measurement is used when available.


In [ ]:
        if "Qto_SpaceBaseQuantities" in psets:
            qto = psets["Qto_SpaceBaseQuantities"]
            area = qto.get("NetFloorArea")
            source = "Qto_SpaceBaseQuantities"
            area_type = "Net Floor Area"    

            if area is None:
                logging.info("NetFloorArea not found, trying GrossFloorArea")
                area = qto.get("GrossFloorArea")  # fallback gross
                if area is not None:
                    source = "Qto_SpaceBaseQuantities (Gross)"
                    area_type = "Gross Floor Area"


### fallback quantity set
if no area is found in the standard quantity set, try to use a Revit-like Dimensions property set. 
this is a secondary source for the area and is used if Qto_SpaceBaseQuantities is missing. 
if no area is found in either source, log a message indicating that manual checking is required.


In [ ]:
        if area is None and "Dimensions" in psets:
            area = psets["Dimensions"].get("Area")
            if area is not None:
                source = "Dimensions"
                area_type = "Revit Dimensions Area"

        if area is None:
            logging.info("No area found for space id %s in either Qto_SpaceBaseQuantities or Dimensions. Manual check required.",
                space.id())


### get space identification
retrieve a human-readable name for the space. if the LongName property is available, use it; 
otherwise, use the Name property. if neither is present, default to "Unnamed". 
also store the unique IFC id of the space.


In [ ]:
        name = getattr(space, "LongName", None) or getattr(space, "Name", "Unnamed")
        space_id = space.id()


### print and append results
if verbose mode is enabled, print the space id, name, and area in a readable format. 
append a dictionary with all relevant information about this space to the results list, 
including the id, name, area, source of area, and area type. 
finally, after processing all spaces, return the complete list of space dictionaries.


In [ ]:
        if verbose:
            print(
                f"Space ID: {space_id} | "
                f"Name: {name} | "
                f"Area: {area if area is not None else 'No area found'} m²"
            )

        results.append({
            "id": space_id,
            "name": name,
            "area": area,
            "source": source,
            "area_type": area_type
        })

    return results
